# Euchre
Goal: code up a playable euchre game to better understand winning probabilities of different starting hands  
Given: Right, Ace, Queen, off Ace, off ten; 3-suited, dealer spot - what's the likelihood of winning each number of tricks that round?  
Genetic Algorithm to determine next play would be cool - start with random strategy, adapt to new weights as games progress  
Streamlit app where user can select cards in hand and table position and get breakdown of likelihood of taking 5/4/3/2/1/0 tricks

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys

sys.path.insert(0, 'C:/Users/jerem/Desktop/nonsense/euchre/')

In [2]:
from utils import EuchreGame

In [3]:
euchre_game = EuchreGame()

In [4]:
euchre_game.score

{'t1': 0, 't2': 0}

In [5]:
euchre_game.dealer

'p1'

In [6]:
euchre_game.next_to_deal

['p2', 'p3', 'p4', 'p1']

In [7]:
euchre_game.print_score()

Current score: 0-0


In [8]:
euchre_game.card_suits

['S', 'C', 'H', 'D']

In [9]:
euchre_game.card_values

['A', 'K', 'Q', 'J', 'T', '9']

In [10]:
deck_of_cards = euchre_game.shuffle_deck_of_cards()
for card in deck_of_cards:
    print(card, end=',')

9_S,9_C,9_H,9_D,T_S,T_C,T_H,T_D,J_S,J_C,J_H,J_D,Q_S,Q_C,Q_H,Q_D,K_S,K_C,K_H,K_D,A_S,A_C,A_H,A_D,

### Deal cards

In [20]:
player_hands, card_flipped_up = euchre_game.deal_hand()

None


In [21]:
player_hands

{'p1': ['K_S', 'J_S', 'Q_S', 'T_C', 'J_H'],
 'p2': ['A_H', 'Q_C', 'A_D', 'T_D', 'K_D'],
 'p3': ['Q_H', 'K_C', 'A_C', '9_S', 'A_S'],
 'p4': ['9_C', '9_D', 'Q_D', 'T_H', '9_H']}

In [22]:
card_flipped_up

'T_S'

### Choose suit for trump

In [23]:
if euchre_game.eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S']):
    print('order up trump')

In [24]:
if euchre_game.eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_S', 'J_C', 'J_S']):
    print('order up trump')

order up trump


In [25]:
euchre_game.choose_open_trump(hand=['K_S', 'A_S', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

'S'

In [26]:
euchre_game.choose_open_trump(hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

In [27]:
calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                           player_hands=player_hands)

None


In [28]:
print(player_hands)
print(calling_player)
print(trump)

{'p1': ['K_S', 'J_S', 'Q_S', 'T_C', 'J_H'], 'p2': ['A_H', 'Q_C', 'A_D', 'T_D', 'K_D'], 'p3': ['Q_H', 'K_C', 'A_C', '9_S', 'A_S'], 'p4': ['9_C', '9_D', 'Q_D', 'T_H', '9_H']}
p1
S


### Play trick

In [29]:
cards_in_play, player_led = euchre_game.play_trick(player_hands=player_hands, 
                                       trump=trump, 
                                       next_to_play_list=euchre_game.next_to_deal, 
                                       verbose=True)

Player p2 plays A_H, Player p3 plays A_S, Player p4 plays 9_H, Player p1 plays J_H, 

In [30]:
cards_in_play

{'p2': 'A_H', 'p3': 'A_S', 'p4': '9_H', 'p1': 'J_H'}

In [31]:
player_led

'p2'

In [32]:
euchre_game.determine_trick_winner(cards_in_play={'p2': '9_S', 'p3': 'J_C', 'p4': 'J_H', 'p1': 'A_S'}, 
                           trump='S', 
                           player_led=player_led, verbose=True)

p3 wins trick


'p3'

In [33]:
euchre_game.determine_trick_winner(cards_in_play={'p2': '9_S', 'p3': 'J_C', 'p4': 'J_S', 'p1': 'A_S'}, 
                           trump='D', 
                           player_led=player_led)

None


'p1'

In [ ]:
trick_winner = euchre_game.determine_trick_winner(cards_in_play=cards_in_play,
                                           trump=trump,
                                           player_led=player_led)

In [ ]:
trick_winner

In [54]:
def swap_dealer_card(card_flipped_up: str,
                     dealer_hand: list, 
                     verbose=False) -> list:
    """
    Function to swap out ordered up trump card with one from dealer's hand
    Returns: dealer new hand

    """
    # TODO: apply some strategy here - drop lowest non-trump card
    # remove first card from dealer hand
    removed_card = dealer_hand[0]
    dealer_hand = dealer_hand[1:]
    dealer_hand.append(card_flipped_up)
    print(f'Dealer discards {removed_card} and picks up {card_flipped_up}' if verbose else None)
    return dealer_hand

In [57]:
new_dealer_hand = swap_dealer_card(card_flipped_up='A_S', dealer_hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'])
new_dealer_hand

['A_D', '9_H', 'K_S', 'J_D', 'A_S']

In [60]:
new_dealer_hand = euchre_game.swap_dealer_card(card_flipped_up='A_S', dealer_hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'], 
                                               verbose=True)
new_dealer_hand

Dealer discards J_H and picks up A_S


['A_D', '9_H', 'K_S', 'J_D', 'A_S']

### Play card

In [ ]:
euchre_game.play_card(hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'], 
          trump='S',
          cards_in_play=['K_H', 'T_H'], 
          suit_led='H')

In [ ]:
def play_card(hand,
              trump,
              cards_in_play,
              suit_led=None):
    """
    Function to return card to play in hand
    TODO: check if partner has winning card_in_pot
    """
    card_values = {
        '9': 1,
        'T': 2,
        'J': 3,
        'Q': 4,
        'K': 5,
        'A': 6
    }
    # play last card
    if len(hand) == 1:
        return hand[0]
    
    # lead card
    if len(cards_in_play) < 1:
        for idx, card in enumerate(hand):
            # 1 - play right bauer
            if card[-1] == trump and card[0] == 'J':
                return hand[idx]
            # 2 - play off ace
            elif card[-1] != trump:
                if card[0] == 'A':
                    return hand[idx]
            # 3 - TODO: else play highest non-trump card
        return hand[0]
        
    # follow suit
    if suit_led is not None:
        # play lowest card in the suit played
        card_to_play_points = -1
        idx_to_return = -1
        for idx, card in enumerate(hand):
            if card[-1] == suit_led:
                card_points = card_values[card[0]]
                if card_points < card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        return hand[idx_to_return]
    
    # play other highest non-trump card
    else:
        card_to_play_points = -1
        idx_to_return = -1
        for idx, card in enumerate(hand):
            if card[-1] != trump:
                card_points = card_values[card[0]]
                if card_points > card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        if idx_to_return > -1:
            return hand[idx_to_return]
        
        else:  # only has trump left, play lowest trump
            trump_card_points = 9
            for idx, card in enumerate(hand):
                card_points = card_values[card[0]]
                if card_points < trump_card_points:
                    trump_card_points = card_points
                    idx_to_return = idx
            return hand[idx_to_return]


In [ ]:
play_card(hand=['9_D', 'T_C', '9_C', 'J_H', 'A_C'],
          trump='S',
          cards_in_play=[],
          suit_led=None)

In [ ]:
play_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'],
          trump='S',
          cards_in_play=[],
          suit_led=None)

In [ ]:
play_card(hand=['9_D', 'T_C', '9_C', 'J_S', 'A_C'],
          trump='S',
          cards_in_play=['K_C'],
          suit_led=None)

### Play full game

In [67]:
%%time

euchre_game = EuchreGame()
euchre_game.play_full_game(verbose=True)

Hand #0- Dealer: p1; Card flipped up: J_C
Player p3 has chosen D as trump
Player p2 plays A_S, Player p3 plays A_D, Player p4 plays J_D, Player p1 plays K_C, p4 wins trick
Player p4 plays K_D, Player p1 plays 9_H, Player p2 plays T_C, Player p3 plays A_C, p4 wins trick
Player p4 plays T_S, Player p1 plays Q_H, Player p2 plays J_S, Player p3 plays 9_D, p3 wins trick
Player p3 plays J_H, Player p4 plays 9_C, Player p1 plays T_H, Player p2 plays A_H, p3 wins trick
Player p3 plays T_D, Player p4 plays Q_C, Player p1 plays 9_S, Player p2 plays K_H, p3 wins trick
Trick winners: {'p2': 0, 'p3': 3, 'p4': 2, 'p1': 0}
t1 scores 1
Current score: 1-0
Hand #1- Dealer: p2; Card flipped up: 9_S
Player p2 has chosen C as trump
Player p3 plays A_D, Player p4 plays K_C, Player p1 plays A_H, Player p2 plays T_D, p4 wins trick
Player p4 plays A_S, Player p1 plays 9_H, Player p2 plays 9_C, Player p3 plays A_C, p3 wins trick
Player p3 plays J_H, Player p4 plays Q_D, Player p1 plays K_S, Player p2 plays T_C,

In [ ]:
euchre_game.score

In [ ]:
cards_in_play

In [ ]:
cards_in_play.values()

In [ ]:
all_cards_played = []
all_cards_played.append(value for value in cards_in_play.values())

In [ ]:
all_cards_played

#### Profile performance

In [73]:
%%prun

euchre_game = EuchreGame()
euchre_game.play_full_game(verbose=True)

Hand #0- Dealer: p1; Card flipped up: 9_C
Player p3 has chosen C as trump
Dealer discards T_H and picks up 9_C
Player p2 plays A_D, Player p3 plays A_H, Player p4 plays K_H, Player p1 plays J_H, p2 wins trick
Player p2 plays T_D, Player p3 plays J_C, Player p4 plays K_D, Player p1 plays A_C, p3 wins trick
Player p3 plays T_C, Player p4 plays 9_S, Player p1 plays T_S, Player p2 plays Q_D, p3 wins trick
Player p3 plays Q_C, Player p4 plays K_C, Player p1 plays 9_D, Player p2 plays J_S, p2 wins trick
Player p2 plays 9_H, Player p3 plays K_S, Player p4 plays Q_S, Player p1 plays T_H, p1 wins trick
Trick winners: {'p2': 2, 'p3': 2, 'p4': 0, 'p1': 1}
t1 scores 1
Current score: 1-0
Hand #1- Dealer: p2; Card flipped up: Q_S
Player p2 has chosen S as trump
Dealer discards 9_H and picks up Q_S
Player p3 plays Q_C, Player p4 plays K_S, Player p1 plays J_S, Player p2 plays T_D, p1 wins trick
Player p1 plays 9_C, Player p2 plays 9_S, Player p3 plays Q_H, Player p4 plays J_C, p4 wins trick
Player p4

### Loop through games played

In [76]:
%%time

N_GAMES_TO_PLAY = 100

games_played = 0
t1_wins = 0
for i in range(N_GAMES_TO_PLAY):
    euchre_game = EuchreGame()
    euchre_game.play_full_game(verbose=False)
    games_played += 1
    if euchre_game.score['t1'] > euchre_game.score['t2']:
        t1_wins += 1

None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
N

None- Dealer: p4; None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None
None- Dealer: p1; None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: 

None, None, None, None, None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None- Dealer: p2; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer:

None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None
None- Dealer: p1; None
None
None- Dealer: p2; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
N

None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, N

...

None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, 

None- Dealer: p1; None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None- Dealer: p1; None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, N

None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
No

None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None

None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p2; None
None
None- Dealer: p3; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p4; None
None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None, None, None, None, None
None
None
None- Dealer: p1; None
None
None
None, None, None, N

In [75]:
print(games_played)
print(t1_wins)

875
432


### Next steps for development

In [ ]:
# FIX print None when verbose=False !!!

# build out play card strategy
    # add logic to check if cards in hand are better than all cards played so far
        # check what trump has been played, etc., if highest remaining trump, play that
    # add random strategy to play_card method
# build out strategy on when to call trump
# build out strategy to dealer swapping out card_flipped_up
# track points scored for each hand type for each game
    # cards in hand
    # position - 0,1,2,3 - from next_to_deal
    # player tricks taken
    # team tricks taken
# Handle loners
# Code cleanup
    # add comments to all class methods
    # Add README
    # Add tests to all functions